In [ ]:
!pip install PyMuPDF

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.tree import plot_tree
import numpy as np
import matplotlib.pyplot as plt
import fitz  # PyMuPDF para leitura de PDFs

In [ ]:
# Função para extrair texto de um arquivo PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text += page.get_text()
    return text

# Função para processar o texto extraído e organizar as informações
def process_decision_text(text):
    # Inicializar dicionário para armazenar os dados
    data = {}

    # Extração baseada em palavras-chave
    data['Juiz'] = extract_value(text, 'Juiz:')
    data['Réu'] = extract_value(text, 'Réu:')
    data['Classe Processual'] = extract_value(text, 'Classe Processual:')
    data['Comarca'] = extract_value(text, 'Comarca:')
    data['Decisão Tomada'] = extract_value(text, 'Decisão:')
    data['Pena'] = extract_value(text, 'Pena:')

    return data

# Função auxiliar para localizar valores no texto com base em palavras-chave
def extract_value(text, keyword):
    try:
        start_idx = text.index(keyword) + len(keyword)
        end_idx = text.index('\n', start_idx)  # Fim da linha
        return text[start_idx:end_idx].strip()
    except ValueError:
        return None

# Lista de arquivos PDF
pdf_files = [
    'doc_55662302.pdf',
    'doc_61089526.pdf',
    'doc_61406539.pdf',
    'doc_95012143.pdf',
    'doc_108959255.pdf'
]

# Lista para armazenar os dados extraídos
all_data = []

# Processar cada PDF
for pdf_file in pdf_files:
    text = extract_text_from_pdf(pdf_file)
    decision_data = process_decision_text(text)
    all_data.append(decision_data)

# Criar um DataFrame do pandas
df = pd.DataFrame(all_data)

# Salvar os dados em um arquivo CSV
df.to_csv('decisoes_judiciais.csv', index=False)

# Exibir o DataFrame
print(df)

# Salvar em um arquivo CSV
#df.to_csv("decisoes_judiciais_extracao_finais.csv", index=False)

In [ ]:
# Carregar o dataset
df = pd.read_csv('decisoes_judiciais_finais.csv')

# Exibir as primeiras linhas do dataset para entender seu formato
print("Dados originais:")
print(df.head())

# Transformações no dataset

# 1. Codificar variáveis categóricas usando LabelEncoder
label_encoders = {}
for column in ["Juiz", "Classe Processual", "Comarca", "Decisão Tomada"]:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le  # Guardar os encoders para decodificação futura, se necessário

# 2. Converter a coluna "Pena" para valores numéricos (em meses)
def convert_penalty_to_months(pena):
    if pena == "Nenhuma":
        return 0
    elif "anos" in pena:
        return int(pena.split()[0]) * 12  # Converter anos para meses
    elif "meses" in pena:
        return int(pena.split()[0])  # Manter como meses
    return 0

df["Pena"] = df["Pena"].apply(convert_penalty_to_months)

# 3. Anonimizar a coluna "Réu"
df["Réu"] = df["Réu"].apply(lambda x: f"Réu_{hash(x) % 10000}")  # Gerar identificador único para cada réu

# Exibir as transformações aplicadas
print("\nDados após transformações:")
print(df.head())

# Salvar o dataset transformado para uso posterior
df.to_csv('decisoes_judiciais_anonimizadas.csv', index=False)

df_decisoes = df.copy()

In [ ]:
df_decisoes.describe().round(2)

In [ ]:
# Aplicando Random Forest e visualizando a importância das variáveis
label_encoders = {}
for column in ["Juiz", "Classe Processual", "Comarca", "Decisão Tomada"]:
    le = LabelEncoder()
    df_decisoes[column] = le.fit_transform(df_decisoes[column])
    label_encoders[column] = le

# Definindo os dados de entrada (features) e saída (target)
X = df_decisoes[["Juiz", "Classe Processual", "Comarca", "Pena"]]
y = df_decisoes["Decisão Tomada"]

# Dividindo os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Criando o modelo Random Forest
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_model.fit(X_train, y_train)

# Fazendo previsões
y_pred = random_forest_model.predict(X_test)

# Avaliando a acurácia
accuracy = accuracy_score(y_test, y_pred)

# Extraindo importância das variáveis
feature_importances = random_forest_model.feature_importances_
features = X.columns

importances_df = pd.DataFrame({"Feature": features, "Importance": feature_importances}).sort_values(by="Importance", ascending=False)
importances_df

In [ ]:
# Visualizando a importância das variáveis
plt.figure(figsize=(10, 6))
plt.barh(importances_df['Feature'], importances_df['Importance'], color='blue')
plt.xlabel('Importância')
plt.title('Importância da Variável')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# Plotar as árvores de decisão (apenas as primeiras 3 para demonstração)
plt.figure(figsize=(20, 10))
for i in range(min(3, len(random_forest_model.estimators_))):  # Limitar a 3 árvores ou o número de árvores no modelo
    plt.subplot(1, min(3, len(random_forest_model.estimators_)), i + 1)
    plot_tree(random_forest_model.estimators_[i], feature_names=features, filled=True, rounded=True, class_names=np.unique(y).astype(str))
    plt.title(f"Árvore {i+1}")

plt.tight_layout()
plt.show()